In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# All imports

In [ ]:
import random as rd
import numpy as np
import pandas as pd
import uuid
from random import randint
import concurrent
import matplotlib.pyplot as plt
import random
import heapq
import subprocess

import os
from tqdm.auto import tqdm
import re
import concurrent.futures
import itertools

# Evaluate CA in CPP

In [ ]:
!cp "./drive/MyDrive/CISC455/CAFIT2/ca_fit" "/content"
!cp "./drive/MyDrive/CISC455/NIST_tests/assess" "/content"
!cp -r "./drive/MyDrive/CISC455/NIST_tests/experiments" "/content"
!cp -r "./drive/MyDrive/CISC455/NIST_tests/templates" "/content"
!chmod "755" "./assess"
!chmod "755" "./ca_fit"

In [ ]:
root_directory = './experiments/AlgorithmTesting'
NOT_TEST = []
def extract_proportions_path_version(NOT_TEST,root_directory):
    # Initialize an empty dictionary to store the results
    results_dict = {}

    # Loop through all the folders in the root directory
    for folder_name in os.listdir(root_directory):
        if folder_name in NOT_TEST:
            continue
        folder_path = os.path.join(root_directory, folder_name)
        
        # Check if the folder contains a file named "results.txt"
        results_file_path = os.path.join(folder_path, 'results.txt')
        if os.path.isfile(results_file_path):
            # If the file exists, read its contents and store them in the dictionary
            with open(results_file_path, 'r') as f:
                results_content = f.readlines()
                pvalue_list = []
                if results_content!= None:
                    for pvalue in results_content:
                        pvalue_list.append(float(pvalue.rstrip('\n')))
                results_dict[folder_name] = pvalue_list
    return results_dict

def fitness_calc(NOT_TEST,root_directory):
    pvalue_dict = extract_proportions_path_version(NOT_TEST, root_directory)
    # print(pvalue_dict)
    for key, values in pvalue_dict.items():
        for i, value in enumerate(values):
            if float(value) > 0.01:
                values[i] = 1
            else:
                values[i] = 0
    # print(pvalue_dict)
    for key, values in pvalue_dict.items():
        avg_val = 0
        val_len = len(pvalue_dict[key])
        if val_len != 0:
          avg_val = sum(pvalue_dict[key])/len(pvalue_dict[key])
        pvalue_dict[key] = avg_val
    fitness = 0
    val_len = len(pvalue_dict.values())
    if val_len != 0:
      fitness = sum(pvalue_dict.values()) / val_len
    return fitness
# extract_proportions_path_version(NOT_TEST, root_directory)
fitness_calc(NOT_TEST, root_directory)

0.799099099099099

In [ ]:
def fitness_cpp(seed, cyc_len, grid_size, ca_size, rules:str):
  fileName = str(uuid.uuid4().hex)
  dirName = "./" + fileName
  subprocess.call(["./ca_fit",str(seed), str(cyc_len), 
                   str(grid_size), str(ca_size), 
                   rules, fileName])
  subprocess.call(["mkdir", dirName])
  subprocess.call(["cp", "-r", "./experiments/.", dirName])

  input_length = str(((grid_size*ca_size)**2) * 4)
  # print(input_length)

  fileName = "./"+fileName + ".txt"
  # print(input_length)
  subprocess.call(["./assess", input_length, str(cyc_len / 2), fileName])
  
  file_path = dirName + "/AlgorithmTesting"
  # pre_dict = extract_proportions_path_version(NOT_TEST,root_directory)
  fitness_score = fitness_calc(NOT_TEST, file_path)
  # print(fitness_score)
  subprocess.Popen(["rm", "-rf", dirName])
  subprocess.Popen(["rm", fileName])
  return fitness_score

# these are tests
ca_good = "31 31 31 31 31 63 63 15 63 63 63 31 31 63 63 63 63 63 31 31 31 63 63 63 47 47 63 63 63 63 63 63 15 63 31 63 63 63 63 31 63 63 31 63 63 63 15 31 63 31 31 31 31 63 31 63 63 31 31 31 31 15 31 15"
ca_bad = "234 225 8 104 194 244 111 131 89 96 55 77 95 67 96 238 82 47 156 116 206 52 47 97 184 8 108 43 178 101 147 30 70 119 129 130 146 47 204 61 30 78 96 199 186 146 64 244 41 135 3 173 2 157 117 174 2 143 156 73 97 12"
ca_best_gen = "98 52 177 30 30 72 49 172 2 83 16 123 218 218 46 173 171 137 100 162 138 241 154 13 237 148 163 210 222 222 148 224 134 134 84 226 120 176 189 132 177 170 120 9 177 102 91 225 208 49 184 54 144 208 109 210 47 27 228 47 27 74 239 27"
fitness_cpp(123, 10, 8, 64,ca_best_gen)

0.937138271875114

# CA_grid

In [ ]:
class CA_grid:
    
    def __init__(self, grid_rules, cycle_len = 100, seed = 548234213, size = 64) -> None:
        self.grid_rules = grid_rules
        self.cycle_len = cycle_len
        self.seed = seed
        self.size = size
        self.fitnessValue = 0
        
    def evalFitness(self):
        rules = ''.join([])
        for inner_list in self.grid_rules:
          for rule in inner_list:
            rules += str(rule) + ' '
          rules = rules[:-1] + ' '
        # print(rules)

        result = fitness_cpp(self.seed, self.cycle_len,
                             len(self.grid_rules), self.size, rules.strip())
        
        # print(result)

        return result
                                                 
    def display(self):
        for row in self.grid_rules:
            print(row)
        print()
        rules = ''.join([])
        for inner_list in self.grid_rules:
          for rule in inner_list:
            rules += str(rule) + ' '
          rules = rules[:-1] + ' '
        print(rules)
        print()
CA_grid([[rd.randint(0, 255) for _ in range(8)] for _ in range(8)]).evalFitness()

0.00045045045045045046

# Initilzation

In [ ]:
def permutation(pop_size : int, chrom_size: int)-> list[CA_grid]:
    # return a CA_grid object and each of them contains a rules map
    return [
        CA_grid([
            [rd.randint(0, 255) for _ in range(chrom_size)] for _ in range(chrom_size)
        ]) for _ in range(pop_size)
    ]



# Mutation

In [ ]:
def permutation_swap(individual):
    # Copy the input list to avoid modifying it directly
    new_individual = individual.copy()

    # Swap a pair of rows with uniform probability
    if random.uniform(0, 1) < 0.5:
        row1, row2 = random.sample(range(len(new_individual)), 2)
        new_individual[row1], new_individual[row2] = new_individual[row2], new_individual[row1]

    # Iterate over each row and swap elements with 0.2 probability
    for i in range(len(new_individual)):
        if random.uniform(0, 1) < 0.2:
            col1, col2 = random.sample(range(len(new_individual[i])), 2)
            new_individual[i][col1], new_individual[i][col2] = new_individual[i][col2], new_individual[i][col1]

    return new_individual                                                                                                                                                                                                                                                    

# Parent Selection

In [ ]:
def MPS(fitness, mating_pool_size):
    """Multi-pointer selection (MPS)"""

    selected_to_mate = []

    # student code starts
    fit_sum = sum(fitness)
    number = len(fitness)
    cumulative = 0
    cumulative_prob_dist = []
    for i in range(0, number): # calculate the cumulative probability distribution
        cumulative += fitness[i]
        cumulative_prob_dist.append(float(cumulative/fit_sum))

    current_number = 0
    i = 0
    r = random.uniform(0, 1 / mating_pool_size)
    while current_number < mating_pool_size:
        while r <= cumulative_prob_dist[i]:
            selected_to_mate.append(i)
            r = r + 1 / mating_pool_size
            current_number += 1
        i += 1
    # student code ends
    
    return selected_to_mate


def tournament(fitness, mating_pool_size, tournament_size):
    """Tournament selection without replacement"""

    selected_to_mate = []

    # student code starts
    number = len(fitness)
    for current_number in range(mating_pool_size):
        participants = random.sample(range(number), tournament_size) # ramdomly choose without replacement
        champion = participants[0]
        for i in range(1, tournament_size):
            if fitness[participants[i]] > fitness[champion]:
                champion = participants[i]
        selected_to_mate.append(champion)
    # student code ends
    
    return selected_to_mate


def random_uniform (population_size, mating_pool_size):
    """Random uniform selection"""

    selected_to_mate = []

    # student code starts
    selected_to_mate = random.sample(range(population_size), mating_pool_size)
    # student code ends
    
    return selected_to_mate

# Recommbanation

In [ ]:
def permutation_cut_and_crossfill(list1, list2):
    """Randomly swaps values between two lists."""
    # Determine the length of the lists
    n = len(list1)
    # Choose a random index to split the lists
    split_index = random.randint(1, n-1)
    # Create two sublists from the original lists
    sublist1_1 = list1[:split_index]
    sublist1_2 = list1[split_index:]
    sublist2_1 = list2[:split_index]
    sublist2_2 = list2[split_index:]
    # Shuffle the two sublists from each list
    random.shuffle(sublist1_1)
    random.shuffle(sublist2_1)
    # Combine the shuffled sublists into two new lists
    new_list1 = sublist1_1 + sublist2_2
    new_list2 = sublist2_1 + sublist1_2
    # Return the new lists
    return new_list1, new_list2

# Survivor selection

In [ ]:
def mu_plus_lambda(current_pop, current_fitness, offspring, offspring_fitness):
    # Combine the current population and offspring into a single population
    
    mu = len(current_pop)
    population = current_pop + offspring
    fitness = current_fitness + offspring_fitness

    # Sort the combined population by fitness in descending order
    sorted_pop_and_fit = sorted(zip(population, fitness), key=lambda x: x[1], reverse=True)
    sorted_population = [pop for pop, _ in sorted_pop_and_fit]
    sorted_fitness = [fit for _, fit in sorted_pop_and_fit]

    # Select the top mu individuals as the next generation
    next_pop = sorted_population[:mu]
    next_fitness = sorted_fitness[:mu]

    return next_pop, next_fitness


def replacement(current_pop, current_fitness, offspring, offspring_fitness):
    """replacement selection"""

    population = []
    fitness = []

    # student code starts
    pop_tup = []
    offspring_size = len(offspring_fitness)
    list_pop = list(current_pop).copy()
    list_fit = list(current_fitness).copy()
    list_off = list(offspring).copy()
    list_off_fit = list(offspring_fitness).copy()

    # store as [(fitness,population), (fitness,population), ....]
    for i in range(len(list_fit)):
        pop_tup.append((list_fit[i],list_pop[i]))
    # sort population only tuple list according to fitness
    pop_tup.sort(key=lambda a: a[0],reverse=True)

    # replace the worst n(offspring_size) lambda pop with offspring
    pop_tup = pop_tup[:len(pop_tup)-offspring_size]

    for i in range(offspring_size):
        pop_tup.append((list_off_fit[i],list_off[i]))

    # get first element of tuple list: fitness
    fitness = (list(zip(*pop_tup))[0])
    # get second element of tuple list: population
    population = (list(zip(*pop_tup))[1])
    
    # student code ends
    
    return population, fitness


def random_uniform(current_pop, current_fitness, offspring, offspring_fitness):
    """random uniform selection"""
    population = []
    fitness = []

    # student code starts
    cur_size = len(current_pop)
    os_size = len(offspring)
    total_size = cur_size + os_size
    temp_pop = current_pop + offspring
    temp_fit = current_fitness + offspring_fitness
    random_idx = random.sample(range(total_size - 1), cur_size)
    for i in random_idx:
        population.append(temp_pop[i])
        fitness.append(temp_fit[i])
    
    # student code ends
    
    return population, fitness


# RUN !

In [ ]:
"""
CISC455/851 A1
A genetic algorithm for the eight queens puzzle
"""
def eval_fitness(individual):
    return individual.evalFitness()
    
def generate_offspring(parents, xover_rate, mut_rate):
    parent1, parent2 = parents
    off1, off2 = parent1.grid_rules.copy(), parent2.grid_rules.copy()

    if random.random() < xover_rate:
        off1, off2 = [], []
        for l1, l2 in zip(parent1.grid_rules, parent2.grid_rules):
            off_l1, off_l2 = permutation_cut_and_crossfill(l1, l2)
            off1 = [*off1, off_l1]
            off2 = [*off2, off_l2]
    else:
          off1 = parent1.grid_rules.copy()
          off2 = parent2.grid_rules.copy()

    if random.random() < mut_rate:
        off1 = permutation_swap(off1)
    if random.random() < mut_rate:
        off2 = permutation_swap(off2)

    off1 = CA_grid(off1)
    off2 = CA_grid(off2)

    return off1, off1.evalFitness(), off2, off2.evalFitness()

def main():
    random.seed()
    np.random.seed()

    grid_length = 8 # may extend to N queens
    # you may test on different parameter settings
    popsize = 20
    mating_pool_size = int(popsize*0.5) # has to be even
    tournament_size = 4
    xover_rate = 0.9
    mut_rate = 0.2
    gen_limit = 10

    # # initialize population
    # gen = 0 # initialize the generation counter
    # population = permutation(popsize, grid_length)
    # fitness = []
    # for i in tqdm(range(0, popsize), desc="evaluate population"):
        
    #     fitness.append(population[i].evalFitness())

    gen = 0  # initialize the generation counter
    population = permutation(popsize, grid_length)

    # Create a ThreadPoolExecutor with as many threads as the number of CPU cores
    
    # fitness = [indi.evalFitness() for indi in tqdm(population, desc="evaluating pop")]
    with concurrent.futures.ProcessPoolExecutor() as executor:
      fitness = list(tqdm(executor.map(eval_fitness, population),
                          total=popsize, desc="evaluate population"))

    # print(fitness)
    # return
    print("generation", gen, ": best fitness", max(fitness), 
          "\taverage fitness", sum(fitness)/len(fitness))

    # evolution begins
    for gen in tqdm(range(1, gen_limit + 1), desc="generating"):
        
        # pick parents 
        parents_index = MPS(fitness, mating_pool_size)
        random.shuffle(parents_index)

        # reproduction
        offspring =[]
        offspring_fitness = []
        i= 0 # initialize the counter for parents in the mating pool
        # print(parents_index)
        # offspring are generated using selected parents in the mating pool
        with concurrent.futures.ProcessPoolExecutor() as executor:
          parent_pairs = [(population[parents_index[i]], 
                           population[parents_index[i + 1]]) for i in range(0, len(parents_index) - 1, 2)]
          results = executor.map(generate_offspring, parent_pairs,
                                 itertools.repeat(xover_rate), itertools.repeat(mut_rate))

          for off1, fit1, off2, fit2 in results:
            offspring.extend([off1, off2])
            offspring_fitness.extend([fit1, fit2])

            if len(offspring) >= mating_pool_size:
              break

        # organize the population of next generation
        population, fitness = mu_plus_lambda(population, 
                                              fitness, 
                                              offspring, offspring_fitness)
        # population, fitness = survivor_selection.replacement(population, fitness, offspring, offspring_fitness)
        # population, fitness = survivor_selection.random_uniform(population, fitness, offspring, offspring_fitness)
        
        print("generation", gen, ": best fitness", max(fitness), "average fitness", sum(fitness)/len(fitness))
        
    # evolution ends
    
    #remove the repeast rows here
    # numpy_pop = numpy.array(population)
    # numpy_fit = numpy.array(fitness)
    # _, index = numpy.unique(numpy_pop, axis=0, return_index=True)
    # population = numpy_pop[index].tolist()
    # fitness = numpy_fit[index].tolist()
    # print the final best solution(s)
    k = 0
    for i in range (0, len(fitness)):
        
        #remove the repeating matrix
        if fitness[i] == max(fitness):
            print("best solution", k + 1, population[i].display(), fitness[i])
            # plot_with_text(population[i])
            k = k+1

# end of main
main()


evaluate population:   0%|          | 0/20 [00:00<?, ?it/s]

generation 0 : best fitness 0.2 	average fitness 0.023082309582309585


generating:   0%|          | 0/10 [00:00<?, ?it/s]

generation 1 : best fitness 0.34877826768237724 average fitness 0.08349091993612541
generation 2 : best fitness 0.34877826768237724 average fitness 0.13315975110495656
generation 3 : best fitness 0.34877826768237724 average fitness 0.18884316423357517
generation 4 : best fitness 0.9516926016926017 average fitness 0.24055596794637904
generation 5 : best fitness 0.9576873576873578 average fitness 0.2946255210159321
generation 6 : best fitness 0.963912978912979 average fitness 0.36915247573809223
generation 7 : best fitness 0.9838049385417806 average fitness 0.5563853416814845
generation 8 : best fitness 0.9946995314118603 average fitness 0.8068935052621065
generation 9 : best fitness 0.9991891891891892 average fitness 0.97384944788027
generation 10 : best fitness 0.9991891891891892 average fitness 0.982010959627398
[250, 249, 63, 37, 249, 150, 236, 5]
[43, 163, 19, 165, 200, 146, 10, 26]
[46, 25, 211, 141, 211, 25, 132, 217]
[243, 149, 20, 189, 243, 149, 116, 156]
[63, 231, 225, 156, 226

In [ ]:
numbers = [60, 130, 216, 60, 143, 76, 113, 92]
[147, 237, 213, 116, 4, 105, 4, 119]
[37, 221, 229, 165, 214, 225, 190, 209]
[183, 148, 4, 177, 42, 241, 148, 104]
[143, 134, 23, 229, 112, 117, 46, 46]
[196, 164, 109, 115, 45, 82, 214, 98]
[238, 213, 137, 79, 105, 160, 34, 88]
result = ""

for num in numbers:
    
      result += str(num) + " "

print(result.strip())